In [1]:
pip install pyproj

Note: you may need to restart the kernel to use updated packages.


In [2]:
import csv
import pandas as pd
from shapely.geometry import Point, Polygon
from pyproj import Proj, transform
from multiprocessing import Pool, Manager, cpu_count
from tqdm import tqdm
import numpy as np

In [3]:
df = pd.read_csv("search.csv")

C:\Users\rbors\AppData\Local\Temp\ipykernel_9592\2031741419.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("search.csv")


In [4]:
df_area = df[df['Longitude'].notnull()]

In [5]:
boroughs = {}

v84 = Proj(proj="latlong",towgs84="0,0,0",ellps="WGS84")
v36 = Proj(proj="latlong", k=0.9996012717, ellps="airy",
        towgs84="446.448,-125.157,542.060,0.1502,0.2470,0.8421,-20.4894")
vgrid = Proj(init="world:bng")

def vectorized_convert(df):
    vlon36, vlat36 = vgrid(df['x'].values, 
                           df['y'].values, 
                           inverse=True)
    converted = transform(v36, v84, vlon36, vlat36)
    df['longitude'] = converted[0]
    df['latitude'] = converted[1]
    return df

df = pd.read_csv("london_boroughs.csv")

C:\Users\rbors\anaconda3\lib\site-packages\pyproj\crs\crs.py:141: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  in_crs_string = _prepare_from_proj_string(in_crs_string)


In [6]:
new_df = vectorized_convert(df)
new_df = new_df[['borough', 'longitude', 'latitude']]
new_df.to_csv('london_boroughs_2.csv')

C:\Users\rbors\AppData\Local\Temp\ipykernel_9592\1140381087.py:12: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  converted = transform(v36, v84, vlon36, vlat36)


In [7]:
new_df = new_df[['borough', 'longitude', 'latitude']]
new_df.to_csv('london_boroughs_2.csv')

In [8]:
boroughs = {}

with open('london_boroughs_2.csv', 'r') as f:
    reader = csv.reader(f)
    next(reader)
    for row in reader:
        try:
            tmp, borough, x, y = row[0], row[1], float(row[2]), float(row[3])            
            if borough not in boroughs:
                boroughs[borough] = []
            boroughs[borough].append((x, y))
        except ValueError:
            continue

In [9]:
compiled_polygons = {borough: Polygon(boundary_coords) for borough, boundary_coords in boroughs.items()}

def find_borough(latitude, longitude):
    point_to_check = Point(longitude, latitude)
    for borough, polygon in compiled_polygons.items():
        if point_to_check.within(polygon):
            return borough
    return None

In [10]:
def apply_find_borough(row):
    if pd.notnull(row['Longitude']) and pd.notnull(row['Latitude']):
        borough = find_borough(row['Latitude'], row['Longitude'])
        return borough
    else:
        return None

# Assuming 'df_area' is your main DataFrame and 'new_df' contains the borough boundaries
df_area['boroughs'] = df_area.apply(apply_find_borough, axis=1)

In [13]:
df_area.to_csv('search_boroughs.csv')